In [6]:
import torch
import math

def generate_delay_doppler_channel_parameters_torch(N, M, car_fre, delta_f, T, max_speed, device='cpu'):
    one_delay_tap = 1 / (M * delta_f)
    one_doppler_tap = 1 / (N * T)

    delays = torch.tensor([0, 30, 150, 310, 370, 710, 1090, 1730, 2510], dtype=torch.float32, device=device) * 1e-9
    taps = delays.numel()
    delay_taps = torch.round(delays / one_delay_tap).int()

    pdp = torch.tensor([0, -1.5, -1.4, -3.6, -0.6, -9.1, -7.0, -12.0, -16.9], dtype=torch.float32, device=device)
    pow_prof = 10 ** (pdp / 10)
    pow_prof = pow_prof / pow_prof.sum()

    # complex fading
    real = torch.randn(1, taps, device=device)
    imag = torch.randn(1, taps, device=device)
    chan_coef = torch.sqrt(pow_prof) * math.sqrt(0.5) * (real + 1j * imag)

    max_UE_speed = max_speed * 1000 / 3600
    Doppler_vel = (max_UE_speed * car_fre) / 299792458.0
    max_Doppler_tap = Doppler_vel / one_doppler_tap
    Doppler_taps = max_Doppler_tap * torch.cos(2 * math.pi * torch.rand(1, taps, device=device))

    return chan_coef, delay_taps, Doppler_taps, taps


In [9]:
import numpy as np
import matplotlib.pyplot as plt


# --- Example run ---
N = 64
M = 64
car_fre = 28e9
delta_f = 15e3
T = 1 / delta_f
max_speed = 120  # km/h

chan_coef, delay_taps, Doppler_taps, delays, pdp, pow_prof, taps = \
    generate_delay_doppler_channel_parameters_torch(N, M, car_fre, delta_f, T, max_speed)

# --- Print parameters ---
print("Number of taps:", taps)
print("Delay taps:", delay_taps)
print("Doppler taps:", np.round(Doppler_taps, 2))
print("Complex channel coefficients:")
for i, c in enumerate(chan_coef):
    print(f"Tap {i+1}: |h|={np.abs(c):.3f}, ∠h={np.angle(c, deg=True):.1f}°")


# --- Visualization ---
fig, axs = plt.subplots(1, 3, figsize=(15, 4))

# Power Delay Profile
axs[0].stem(delays * 1e9, 10 * np.log10(pow_prof), use_line_collection=True)
axs[0].set_xlabel("Delay [ns]")
axs[0].set_ylabel("Power [dB]")
axs[0].set_title("EVA Power Delay Profile")
axs[0].grid(True)

# Doppler Tap Distribution
axs[1].stem(range(taps), Doppler_taps, use_line_collection=True)
axs[1].set_xlabel("Tap Index")
axs[1].set_ylabel("Doppler Tap")
axs[1].set_title("Random Doppler Shifts (Jakes Model)")
axs[1].grid(True)

# Channel Coefficient Magnitude & Phase
axs[2].scatter(np.real(chan_coef), np.imag(chan_coef), color='r')
axs[2].axhline(0, color='k', lw=0.5)
axs[2].axvline(0, color='k', lw=0.5)
axs[2].set_xlabel("Real(h)")
axs[2].set_ylabel("Imag(h)")
axs[2].set_title("Complex Channel Coefficients")
axs[2].grid(True)

plt.tight_layout()
plt.show()



ValueError: not enough values to unpack (expected 7, got 4)